ref count baseline 알고리즘 이용하여 predict한 값으로
sorting again 하자

In [1]:
import pandas as pd
import numpy as np
import os
os.listdir('../input/')

['df_ref2_session.csv',
 'new_submission_dwell_time.csv',
 'new_submission_baseline.csv',
 'df_ref.csv',
 'ref_count2_baselineOnly_pred.csv',
 'train.csv',
 'df_ref2.csv',
 'submission_popular.csv',
 'item_metadata.csv',
 'df_full_except_noinfo.csv',
 'test.csv',
 'new_submission.csv',
 'dwell_df.csv',
 'baseline_sorting.csv',
 'dwell_time_agg.csv',
 'test2.csv']

In [107]:
blo_pred = pd.read_csv(path + 'ref_count2_baselineOnly_pred.csv')

In [121]:
path = '../input/'
new_submission = pd.read_csv(path + 'new_submission_baseline.csv')
print(new_submission.shape)
new_submission.head(3)

(121526, 7)


,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,1050068 2059240 3241426 2033381 1724779 127131...,1050068 2059240 3241426
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,2714480 4476010 3843244 3833012 9017890 198100...,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,2817590 3565720 2947584 4115018 2039671 383653...,2817590


In [122]:
new_submission.impressions = new_submission.impressions.apply(lambda x : x.split(' '))

In [109]:
blo_pred = blo_pred.groupby('session_id').apply(lambda x : x.sort_values(['est'], ascending = False)).reset_index(drop = True)
blo_pred.iid = blo_pred.iid.apply(str)
blo_pred = blo_pred.groupby('session_id')['iid'].apply(lambda x : ' '.join(x)).to_frame().reset_index()
blo_pred.iid = blo_pred.iid.apply(lambda x : x.split(' '))
blo_pred.head(3)

,session_id,iid
0,0000e0500c2e7,"[3793522, 8342182, 9377514, 2605288, 7354306, ..."
1,000177b850519,"[17529, 16831, 2759696, 153594, 9117602, 17579..."
2,00017b3b2c136,"[2810449, 1101646, 509856, 5717238, 2565452, 4..."


In [123]:
new_submission = pd.merge(new_submission, blo_pred, how = 'left')
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted,iid
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[1050068, 2059240, 3241426, 2033381, 1724779, ...",1050068 2059240 3241426,"[1963235, 4491766, 5162722, 3442272, 1724779, ..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",2817590,NaN


In [124]:
idx = ~new_submission.ref_count_sorted.isnull()
new_submission.loc[idx,'ref_count_sorted'] = new_submission.loc[idx,'ref_count_sorted'].apply(lambda x : x.split(' '))

In [64]:
idx = ~new_submission.iid.isnull()
aa = new_submission.loc[idx,'impressions'].apply(len)
bb = new_submission.loc[idx, 'iid'].apply(len)

In [73]:
len(aa) / len(aa>=bb)

1.0

# fix ref_count

In [142]:
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted,iid
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[1050068, 2059240, 3241426, 2033381, 1724779, ...","[1050068, 2059240, 3241426]","[1963235, 4491766, 5162722, 3442272, 1724779, ..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN,NaN
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",[2817590],NaN


In [152]:
def sort_again(imp, ref_sorted, iid) :
    
    if type(ref_sorted)!= list :
        return imp
    
    if type(iid)!= list :
        return imp
    

    i = 0
    for ref in ref_sorted :
        if ref in imp :
            i+=1
            if ref in iid :
                j = iid.index(ref)
                iid = iid[:j] + iid[j+1:]


    final_list = imp[:i] + iid + list(set(imp[i:]) - set(iid))
    return final_list

In [154]:
new_submission['sort_again'] = new_submission.apply(lambda x : sort_again(x['impressions'], x['ref_count_sorted'], x['iid']), axis = 1)

In [155]:
new_submission.head(3)

,user_id,session_id,timestamp,step,reference,impressions,ref_count_sorted,iid,sort_again
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[1050068, 2059240, 3241426, 2033381, 1724779, ...","[1050068, 2059240, 3241426]","[1963235, 4491766, 5162722, 3442272, 1724779, ...","[1050068, 2059240, 3241426, 1963235, 4491766, ..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...",NaN,NaN,"[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[2817590, 3565720, 2947584, 4115018, 2039671, ...",[2817590],NaN,"[2817590, 3565720, 2947584, 4115018, 2039671, ..."


In [156]:
def mrr(answer, imp) :
    myscore = 0
    
    for i,j in zip(answer, imp) :
        if i in j :
            myscore += 1/(j.index(i) + 1)
    print(myscore / len(imp))

In [157]:
new_submission.reference = new_submission.reference.apply(str)
mrr(new_submission['reference'], new_submission['sort_again'])

0.7669310355806487


# defalut submission file + baselineOnly predict

In [163]:
default_sub = pd.read_csv(path + 'new_submission.csv')
default_sub.impressions = default_sub.impressions.apply(lambda x : x.split('|'))
default_sub.reference = default_sub.reference.apply(str)
default_sub.head(3)

,user_id,session_id,timestamp,step,reference,impressions
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ..."


In [164]:
mrr(default_sub['reference'], default_sub['impressions'])

0.4517556722779725


In [173]:
aa = pd.merge(default_sub, blo_pred, how = 'left')
aa.loc[aa.iid.isnull(), 'iid'] = aa.loc[aa.iid.isnull(), 'impressions']
aa.head(3)

,user_id,session_id,timestamp,step,reference,impressions,iid
0,004A07DM0IDW,1d688ec168932,1541555792,6,3241426,"[2059240, 2033381, 1724779, 127131, 399441, 10...","[1963235, 4491766, 5162722, 3442272, 1724779, ..."
1,00Y1Z24X8084,26b6d294d66e7,1541651766,1,3843244,"[2714480, 4476010, 3843244, 3833012, 9017890, ...","[2714480, 4476010, 3843244, 3833012, 9017890, ..."
2,01V3WDTDM5CU,07628a0f5be0b,1541575516,4,2817590,"[3565720, 2947584, 4115018, 2039671, 3836538, ...","[3565720, 2947584, 4115018, 2039671, 3836538, ..."


In [174]:
mrr(aa['reference'], aa['iid'])

0.3504994501249227


# 예측간으한 값 only

In [180]:
aa = pd.merge(default_sub, blo_pred, how = 'left')
idx = ~aa.iid.isnull()
aa = aa[idx]

In [177]:
print('ref count로 예측 가능한 값들 default : ' ,mrr(aa.loc[idx,'reference'], aa.loc[idx,'impressions']))
print('ref count로 예측 가능한 값들 sorting : ' ,mrr(aa.loc[idx,'reference'], aa.loc[idx,'iid']))

0.3933998347533362
ref count로 예측 가능한 값들 default :  None
0.2447909238036744
ref count로 예측 가능한 값들 sorting :  None
